In [1]:
# type: ignore
try:
    import google.colab  
    from google.colab import output

    COLAB = True
    %pip install sae-lens transformer-lens sae-dashboard
except:
    COLAB = False
    from IPython import get_ipython  # type: ignore

    ipython = get_ipython()
    assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

# Standard imports
import os
import torch


# Imports for displaying vis in Colab / notebook
import webbrowser
import http.server
import socketserver
import threading

PORT = 8000

torch.set_grad_enabled(False);

device = "cuda" if torch.cuda.is_available() else "cpu"
from univ_utils import load_model, load_sae, load_data, get_batch

/home/zara-ss/Documents/feature_universality/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# For the most part I'll try to import functions and classes near where they are used
# to make it clear where they come from.

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

Device: cuda


In [3]:
def display_vis_inline(filename: str, height: int = 850):
    """
    Displays the HTML files in Colab. Uses global `PORT` variable defined in prev cell, so that each
    vis has a unique port without having to define a port within the function.
    """
    if not (COLAB):
        webbrowser.open(filename)

    else:
        global PORT

        def serve(directory):
            os.chdir(directory)

            # Create a handler for serving files
            handler = http.server.SimpleHTTPRequestHandler

            # Create a socket server with the handler
            with socketserver.TCPServer(("", PORT), handler) as httpd:
                print(f"Serving files from {directory} on port {PORT}")
                httpd.serve_forever()

        thread = threading.Thread(target=serve, args=("/content",))
        thread.start()

        output.serve_kernel_port_as_iframe(
            PORT, path=f"/{filename}", height=height, cache_in_notebook=True
        )

        PORT += 1

In [4]:
model_name = "8-768"
sae_name = "443ngubm"

model = load_model(model_name=model_name, device=device, ckpt_iter=None)
sae = load_sae(sae_name=sae_name, device=device, ckpt_iter=None, model_name=model_name)

/home/zara-ss/Documents/feature_universality/univ_utils.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_file, map_location=device)
/home/za

In [5]:
data, _ = load_data(dataset="openwebtext", device=device)

# TODO: ideally want to do more than 10k tokens
owt_tokens, _ = get_batch(data, block_size=sae.cfg.context_size, batch_size=int(1e5))

(owt_tokens.shape[0] * owt_tokens.shape[1]) / 1e8

In [9]:
mask_features = [29, 45, 76, 114, 128, 132, 164, 166, 186, 194]
topk_features = [2507, 8124, 4152, 6121, 2805, 6802, 2473, 4656,  963, 5231, 3548, 5340,
        4637, 3778, 7321, 4838, 7141, 1123, 4389, 4815, 6073, 6927, 3719, 7930,
        2893, 5147, 4164, 3066, 7967, 1919, 2553, 5472,  125, 2128,  835, 4126,
        2708, 8187, 6101, 3574]
features = topk_features

In [11]:
from sae_dashboard.sae_vis_data import SaeVisConfig
feature_vis_config_gpt = SaeVisConfig(
    hook_point=sae.cfg.hook_name,
    features=topk_features,
    minibatch_size_features=64,
    minibatch_size_tokens=256,
    verbose=True,
    device=device,
)

from sae_dashboard.sae_vis_runner import SaeVisRunner
visualization_data_gpt = SaeVisRunner(feature_vis_config_gpt).run(
    encoder=sae,  # type: ignore
    model=model,
    tokens=owt_tokens,  # type: ignore
)

type(visualization_data_gpt)

Forward passes to cache data for vis: 100%|██████████| 391/391 [05:16<00:00,  1.38it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 15.26 GiB. GPU 0 has a total capacity of 23.64 GiB of which 5.08 GiB is free. Including non-PyTorch memory, this process has 18.54 GiB memory in use. Of the allocated memory 16.69 GiB is allocated by PyTorch, and 1.41 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
from sae_dashboard.data_writing_fns import save_feature_centric_vis
filename_fn = lambda features : f"dashboards/dashboards_{sae_name}_{'_'.join(map(str, features))}.html"
filename = filename_fn(features)
save_feature_centric_vis(sae_vis_data=visualization_data_gpt, filename=filename)
display_vis_inline(filename=filename)

/home/zara-ss/Documents/feature_universality/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'features' is not defined